## 필요 라이브러리 호출

In [104]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import time

# 셀레니움(selenium)으로 브라우저를 컨트를하기 위해서는 webdriver를 설치해야 한다
# 구글크롬은 chromedriver로 검색하면 exe파일을 다운로드 할 수 있다.
# webdriver 설치 위치를 정의한다.
browser = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver')

## 데이터 크롤링

In [108]:
def crawling(url):
    # 셀레니움(selenium)으로 브라우저를 컨트를하기 위해서는 webdriver를 설치해야 한다
    # 구글크롬은 chromedriver로 검색하면 exe파일을 다운로드 할 수 있다.
    # webdriver 설치 위치를 정의한다.
    browser = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver')
    # 불러오고자하는 url을 .get으로 호출한다.
    browser.get(url)
    
    # page_source파라미터를 이용하면 HTML정보를 가지고 온다.
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')   # 뷰티풀숩으로 HTML을 파싱하고 필요한 데이터 수집
    table_tags = soup.find_all("table")   # find_all함수를 이용하면, TABLE태그로 지정된 곳만 뽑아서, 배열 형태로 저장
    table = table_tags[0]    # html상에서 table 순서, 첫번째 테이블 가지고 와야 함.
    p=parser.make2d(table)

    df_total=pd.DataFrame(p[1:],columns=p[0])    # 데이터 프레임으로 저장

    num = soup.find_all("button")[5].text   # 5번째 button에 페이지수 정보가 있음
    for j in range(1, int(num)):
        btn = browser.find_element_by_class_name('paginationWidget-next')
        btn.click()     # 버튼 클릭
        time.sleep(5)

        # 2페이지 이후 데이터 병합
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table_tags = soup.find_all("table")
        table = table_tags[0]
        p = parser.make2d(table)

        df = pd.DataFrame(p[1:], columns=p[0])
        df_total = pd.concat([df_total, df], 0)
        
    return df_total
        # 엑셀로 다운로드
        # df_total.to_excel("test.xlsx")

In [109]:
list_url = ["http://mlb.mlb.com/stats/sortable.jsp?c_id=mlb#elem=%5Bobject+Object%5D&tab_level=child&click_text=Sortable+Player+hitting&game_type='R'&season=2019&season_type=ANY&league_code='MLB'&sectionType=sp&statType=hitting&page=1&ts=1580532864437&playerType=ALL",
"http://mlb.mlb.com/stats/sortable.jsp?c_id=mlb#elem=%5Bobject+Object%5D&tab_level=child&click_text=Sortable+Player+fielding&game_type='R'&season=2019&season_type=ANY&league_code='MLB'&sectionType=sp&statType=fielding&page=1&ts=1580539033522&playerType=ALL"]

df_total_hit = crawling(list_url[0])
df_total_field = crawling(list_url[1])

In [113]:
df_total_hit

,RK,Player,Team,,▲,Pos,G▼,AB▼,R▼,H▼,...,SAC▼,SF▼,TB▼,XBH▼,GDP▼,GO▼,AO▼,GO_AO▼,NP▼,PA▼
0,1,"Alaniz, R",CIN,,595798,P,8,1,0,1,...,0,0,1,0,0,0,0,-.--,5,1
1,1,"Brebbia, J",STL,,605154,P,63,1,1,1,...,0,0,1,0,0,0,0,-.--,1,1
2,1,"Drake, O",TB,,543118,P,3,1,0,1,...,0,0,1,0,0,0,0,-.--,4,1
3,1,"Liriano, F",PIT,,434538,P,67,1,0,1,...,0,0,1,0,0,0,0,-.--,15,2
4,1,"Lugo, S",NYM,,607625,P,57,1,0,1,...,0,0,1,0,0,0,0,-.--,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,984,"Yates, K",SD,,489446,P,59,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
33,984,"Ynoa, G",BAL,,593679,P,4,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
34,984,"Ynoa, H",ATL,,660623,P,2,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
35,984,"Zamora, D",NYM,,623354,P,17,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0


In [111]:
df_total_field

,RK,Player,Team,,▲,Pos,G▼,GS▼,INN▼,TC▼,...,A▼,E▼,DP▼,SB▼,CS▼,SBPCT▲,PB▼,C_WP▼,FPCT▼,RF▼
0,1,"Abreu, B",HOU,,650556,P,7,0,8.2,4,...,2,0,0,-,-,-,-,-,1.000,0.57
1,1,"Adames, C",SF,,542436,SS,7,5,46.0,15,...,8,0,2,-,-,-,-,-,1.000,2.14
2,1,"Adams, A",DET,,542866,P,15,0,16.2,4,...,3,0,0,-,-,-,-,-,1.000,0.27
3,1,"Adams, C",NYY,,664856,P,13,0,25.1,2,...,2,0,0,-,-,-,-,-,1.000,0.15
4,1,"Adduci, J",CHC,,451192,RF,1,1,8.0,5,...,0,0,0,-,-,-,-,-,1.000,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1319,"Walker, T",ARI,,592836,P,1,1,1.0,0,...,0,0,0,-,-,-,-,-,-,0.00
1,1319,"Warren, A",SEA,,605521,P,6,0,5.1,0,...,0,0,0,-,-,-,-,-,-,0.00
2,1319,"Williams, A",WSH,,657117,P,2,0,0.1,0,...,0,0,0,-,-,-,-,-,-,0.00
3,1319,"Williams, D",MIL,,642207,P,13,0,13.2,0,...,0,0,0,-,-,-,-,-,-,0.00


### 전처리 진행
- 불필요한 컬럼 삭제, RK, Team, 빈 컬럼, 등
- hit & field data mergy : key column Player & Team $ Position
- 1B, 2B, 3B, SS 포지션만 추출
- AB 타석수 및 수비 차여수가 10번 이상의 선수만 대상으로 한다.


In [148]:
# 컬럼 rename
df_total_hit = df_total_hit.rename(columns={'RK':'RK', 'Player':'Player', 'Team':'Team', 
                                            '':'del', '▲':'del2', 'Pos':'Pos', 'G▼':'G', 'AB▼':'AB', 'R▼':'R',
                                            'H▼':'H', '2B▼':'2B', '3B▼':'3B', 'HR▼':'HR',
                                            'RBI▼':'RBI', 'BB▼':'BB', 'SO▼':'SO', 'SB▼':'SB',
                                            'CS▼':'CS', 'AVG▼':'AVG', 'OBP▼':'OBP',
                                            'SLG▼':'SLG', 'OPS▼':'OPS', 'IBB▼':'IBB',
                                            'HBP▼':'HBP', 'SAC▼':'SAC', 'SF▼':'SF', 
                                            'TB▼':'TB', 'XBH▼':'XBH', 'GDP▼':'GDP',
                                            'GO▼':'GO', 'AO▼':'AO', 'GO_AO▼':'GO_AO',
                                            'NP▼':'NP', 'PA▼':'PA'})
df_total_field = df_total_field.rename(columns={'RK':'RK', 'Player':'Player', 'Team':'Team', '':'del',
                                            '▲':'del2', 'Pos':'Pos', 'G▼':'G', 'GS▼':'GS',
                                            'INN▼':'INN', 'TC▼':'TC', 'PO▼':'PO', 'A▼':'A',
                                            'E▼':'E', 'DP▼':'DP', 'SB▼':'SB', 'CS▼':'CS',
                                            'SBPCT▲':'SBPCT', 'PB▼':'PB', 'C_WP▼':'C_WP',
                                            'FPCT▼':'FPCT', 'RF▼':'RF'})

Index(['RK', 'Player', 'Team', 'del', 'del2', 'Pos', 'G', 'AB', 'R', 'H', '2B',
       '3B', 'HR', 'RBI', 'BB', 'SO', 'SB', 'CS', 'AVG', 'OBP', 'SLG', 'OPS',
       'IBB', 'HBP', 'SAC', 'SF', 'TB', 'XBH', 'GDP', 'GO', 'AO', 'GO_AO',
       'NP', 'PA'],
      dtype='object')

In [151]:
# 데이터 백업
player_hit_data = df_total_hit
player_field_data = df_total_field

In [159]:
# 불필요한 컬럼 삭제
player_hit_data = player_hit_data.drop(['RK', 'del', 'del2'], axis=1)
player_field_data = player_field_data.drop(['RK', 'del', 'del2'], axis=1)

In [167]:
# column Player & Team $ Position 기준 병합
df_total = pd.merge(player_hit_data, player_field_data, on=['Player', 'Team', 'Pos']) # how='outer')
df_total
#df_total.to_excel("test.xlsx")

,Player,Team,Pos,G_x,AB,R,H,2B,3B,HR,...,A,E,DP,SB_y,CS_y,SBPCT,PB,C_WP,FPCT,RF
0,"Alaniz, R",CIN,P,8,1,0,1,0,0,0,...,2,1,0,-,-,-,-,-,.800,0.33
1,"Brebbia, J",STL,P,63,1,1,1,0,0,0,...,6,0,0,-,-,-,-,-,1.000,0.11
2,"Drake, O",TB,P,3,1,0,1,0,0,0,...,2,0,0,-,-,-,-,-,1.000,0.16
3,"Liriano, F",PIT,P,67,1,0,1,0,0,0,...,13,0,0,-,-,-,-,-,1.000,0.22
4,"Lugo, S",NYM,P,57,1,0,1,0,0,0,...,6,0,0,-,-,-,-,-,1.000,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1263,"Yates, K",SD,P,59,0,0,0,0,0,0,...,5,0,1,-,-,-,-,-,1.000,0.18
1264,"Ynoa, G",BAL,P,4,0,0,0,0,0,0,...,8,2,0,-,-,-,-,-,.882,0.42
1265,"Ynoa, H",ATL,P,2,0,0,0,0,0,0,...,1,0,0,-,-,-,-,-,1.000,1.00
1266,"Zamora, D",NYM,P,17,0,0,0,0,0,0,...,0,0,0,-,-,-,-,-,-,0.00


In [166]:
df_total.isnull.count()

AttributeError: 'function' object has no attribute 'count'